In [1]:
import pandas as pd                                         # Import pandas library for data manipulation and analysis
import ast                                                  # Import Abstract Syntax Trees module for parsing Python literals from strings
import json                                                 # Import JSON module for encoding/decoding JSON data
from geopy.distance import geodesic                         # Import geodesic from geopy.distance to calculate distances between coordinates
from spotipy.oauth2 import SpotifyClientCredentials         # Import Spotify credentials manager for API authentication
import spotipy                                              # Import Spotipy library for Spotify API interaction
import tableauserverclient as TSC                           # Import Tableau Server Client for Tableau integration
import pantab as pt                                         # Import Pantab for converting between pandas DataFrames and Tableau Hyper files
import requests                                             # Import Requests library for HTTP requests

In [2]:
# -------------------------------- SETLIST.FM ----------------------------------------# 
# Replace with your own API key and artist MBID
API_KEY = "your_setlist_api_key"  # API key for Setlist.fm (replace with your own key)
ARTIST_MBID = "your_setlist_artist_id"  # MusicBrainz Identifier for the artist

# --------------------------------- SPOTIFY ------------------------------------------# 
# Configure spotipy with your Spotify credentials
client_id = 'your_spotify_client_id'  # Your Spotify Client ID
client_secret = 'your_spotify_client_secret'  # Your Spotify Client Secret

# -------------------------------- TABLEAU API ----------------------------------------# 
# Tableau Server Personal Access Token credentials and connection details
TOKENNAME = 'token_name'  # Name of the Tableau PAT
TOKENVALUE = 'token_value'  # Token value (keep secure)
CONTENTURL = 'your_content_url'  # Tableau site content URL
SERVER = 'your_site'  # Tableau Cloud URL

PART I – Hitting the Setlist.Fm API to dig up tour data.
Heads up: the raw stuff is still messy — we’re just getting started 🤘

In [3]:
def fetch_all_setlists(mbid: str, api_key: str, max_pages: int = 100):
    """
    Fetches all available setlists for a given artist from the Setlist.fm API.

    Args:
        mbid (str): MusicBrainz Identifier for the artist.
        api_key (str): Your Setlist.fm API key.
        max_pages (int, optional): Maximum number of pages to fetch. Defaults to 100.

    Returns:
        pd.DataFrame: A DataFrame containing all retrieved setlists, normalized for easy analysis.

    Notes:
        - The function respects API limits with a short delay between requests.
        - If an error occurs (e.g., wrong API key or MBID), it logs the issue and stops.
    """
    base_url = f"https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists"  # Constructs the API endpoint URL using the artist's MusicBrainz ID
    headers = {
        "Accept": "application/json",  # Specifies that we want the response in JSON format
        "x-api-key": api_key  # Adds the API key to the request headers for authentication
    }

    all_setlists = []  # Initialize an empty list to store all setlists
    page = 1  # Start with the first page of results

    while True:  # Begin an infinite loop that will break when conditions are met
        print(f"📄 Fetching page {page}...")  # Print status message with page number
        params = {"p": page}  # Set up the page parameter for pagination
        response = requests.get(base_url, headers=headers, params=params)  # Make HTTP GET request to the API

        if response.status_code != 200:  # Check if the request was not successful
            print(f"❌ Error: {response.status_code} - {response.text}")  # Print error message with details
            break  # Exit the loop if an error occurs

        data = response.json()  # Parse the JSON response
        setlists = data.get('setlist', [])  # Extract the setlists array, defaulting to empty list if not found

        if not setlists:  # Check if no setlists were returned
            print("✅ No more setlists found.")  # Print completion message
            break  # Exit the loop if no more setlists are available

        all_setlists.extend(setlists)  # Add the fetched setlists to our collection

        # Handle pagination info from response
        total = int(data.get('total', 0))  # Get total number of items available
        items_per_page = int(data.get('itemsPerPage', 20))  # Get number of items per page, default to 20
        max_page_count = (total // items_per_page) + (1 if total % items_per_page else 0)  # Calculate the total number of pages

        # Stop if last page reached or max_pages limit
        if page >= max_page_count or page >= max_pages:  # Check if we've reached the last page or our limit
            break  # Exit the loop if either condition is met

        page += 1  # Increment the page counter for the next iteration
        time.sleep(0.2)  # Add a small delay to respect API rate limits

    # Convert list of setlists to pandas DataFrame
    return pd.json_normalize(all_setlists)  # Convert the nested JSON data to a flat pandas DataFrame



# 📊 Fetch setlists and show first few rows
df = fetch_all_setlists(ARTIST_MBID, API_KEY)  # Call the function to get all setlists for the specified artist
print(df.head())  # Display the first 5 rows of the resulting DataFrame


📄 Fetching page 1...
❌ Error: 403 - {"message":"Forbidden"}

Empty DataFrame
Columns: []
Index: []


Pulled the tracks from a separate setlist dictionary.
Digging deeper to match the chaos with real songs 

In [4]:
# List to store the results
resultados = []  # Initialize an empty list to collect extracted song data

# Iterate over each row in the DataFrame
for index, row in df.iterrows():  # Loop through each row in the DataFrame with its index
    id_valor = row['id']  # Extract the ID value from the current row
    
    try:
        # Attempt to convert the string representation of a dictionary/list into a Python object
        sets_data = None  # Initialize variable to hold the parsed sets data
        
        # Check if the value is a string and try to parse it
        if isinstance(row['sets.set'], str):  # Check if the 'sets.set' value is a string
            try:
                # First, try parsing as JSON
                sets_data = json.loads(row['sets.set'])  # Attempt to parse the string as JSON
            except json.JSONDecodeError:  # Handle JSON parsing errors
                try:
                    # If JSON fails, try using ast.literal_eval as a fallback
                    sets_data = ast.literal_eval(row['sets.set'])  # Use Python's abstract syntax tree parser as fallback
                except:  # Handle any parsing errors from ast.literal_eval
                    # If both parsing attempts fail, print error and skip to next row
                    print(f"Error processing row {index}, ID: {id_valor}")  # Log the error
                    continue  # Skip to the next row in the DataFrame
        else:
            # If it's already a list or dictionary, assign it directly
            sets_data = row['sets.set']  # Use the value directly if it's already a proper Python object
        
        # Iterate through the main list
        for item in sets_data:  # Loop through each set in the setlist
            # Check if 'song' key exists and is a list
            if 'song' in item and isinstance(item['song'], list):  # Verify that the 'song' field exists and is a list
                for song in item['song']:  # Loop through each song in the set
                    # If the song has a 'name', add it to the results
                    if 'name' in song:  # Check if the song has a name field
                        nome = song['name']  # Extract the song name
                        resultados.append({'id': id_valor, 'nome': nome})  # Add the song ID and name to results list
                    
    except Exception as e:  # Catch any other exceptions that might occur
        # Catch and report any unexpected errors during processing
        print(f"Error processing row {index}, ID: {id_valor}: {str(e)}")  # Log detailed error information

# Create a DataFrame with the results
result_song_id = pd.DataFrame(resultados)  # Convert the collected results into a pandas DataFrame


In [5]:
result_song_id = result_song_id.rename(columns={'nome': 'song'}) #rename the name of the colunm name for song

Uploaded a custom "from-to" table for songs and albums.
Because not every track shows up clean — time to match names and unleash the right album 

In [6]:

df_album_song = pd.read_excel('a7x_albums_songs.xlsx')

In [7]:
df_album_song #FROM-TO about the Songs and Album -- I collected this information manually

,year,Album,Song
0,2001,Sounding the Seventh Trumpet,To End The Rapture
1,2001,Sounding the Seventh Trumpet,Turn The Other Way
2,2001,Sounding the Seventh Trumpet,Darkness Surrounding
3,2001,Sounding the Seventh Trumpet,The Art Of Subconscious Illusion
4,2001,Sounding the Seventh Trumpet,We Come Out At Night
...,...,...,...
84,2023,Life is but a dream,Easier
85,2023,Life is but a dream,G
86,2023,Life is but a dream,(O)rdinary
87,2023,Life is but a dream,(D)eath


In [8]:
df_album_song = df_album_song.rename(columns={'Song': 'song'}) #rename the name of the colunm Song for song
df_album_song = df_album_song.rename(columns={'Album': 'album'}) #rename the name of the colunm Album for album

In [9]:
result_song_id['song_upper'] = result_song_id['song'].str.strip().str.upper()  # Create a new column with the song name stripped of whitespace and converted to uppercase
df_album_song['song_upper'] = df_album_song['song'].str.strip().str.upper()  # Create a new column in the album_song dataframe with song names stripped and uppercased

KeyError: 'song'

PART II – Hit the Spotify API to fetch track links and embed codes.
So you don’t just see the data — you will hear it



In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)  # Set up authentication for Spotify API using client credentials flow
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)  # Create a Spotify API client instance with the authentication manager

def obter_link_spotify(nome_musica, artista="Avenged Sevenfold"):
    """
    Function to search for a song on Spotify and return an embed link
    """
    try:
        # First, try a more specific search
        query = f'track:"{nome_musica}" artist:"{artista}"'  # Create a precise query with track and artist filters
        result = sp.search(q=query, limit=5, type='track')  # Search Spotify API with the query, limiting to 5 tracks
        
        # Check if the search returned any results
        if result['tracks']['items']:  # Check if the search returned any results
            # Look for the closest match
            for item in result['tracks']['items']:  # Loop through each track result
                if artista.lower() in item['artists'][0]['name'].lower():  # Case-insensitive check if the artist matches
                    music_id = item['id']  # Extract the Spotify track ID
                    print(f"Song found: {item['name']} by {item['artists'][0]['name']}")  # Log the found track
                    return f'https://open.spotify.com/embed/track/{music_id}'  # Return the embed URL with the track ID
            
            # If no exact match found, use the first result
            music_id = result['tracks']['items'][0]['id']  # Get the first result's track ID as fallback
            print(f"Closest match: {result['tracks']['items'][0]['name']} by {result['tracks']['items'][0]['artists'][0]['name']}")  # Log the closest match
            return f'https://open.spotify.com/embed/track/{music_id}'  # Return the embed URL with the track ID
        
        # If no results with specific search, try a broader search
        query = f'{nome_musica} {artista}'  # Create a broader query using song name and artist as general search terms
        result = sp.search(q=query, limit=10, type='track')  # Search with broader terms and more results (10)
        
        if result['tracks']['items']:  # Check if the broader search returned any results
            # Look for songs by the requested artist
            for item in result['tracks']['items']:  # Loop through each track in the broader search results
                if artista.lower() in item['artists'][0]['name'].lower():  # Check if any result matches the requested artist
                    music_id = item['id']  # Extract the track ID
                    print(f"Song found in broad search: {item['name']} by {item['artists'][0]['name']}")  # Log the found track
                    return f'https://open.spotify.com/embed/track/{music_id}'  # Return the embed URL
            
            # If specific artist not found, consider it might be a cover or another artist
            music_id = result['tracks']['items'][0]['id']  # Use the first result as a possible cover version
            print(f"Possible cover or different artist: {result['tracks']['items'][0]['name']} by {result['tracks']['items'][0]['artists'][0]['name']}")  # Log the potential cover
            return f'https://open.spotify.com/embed/track/{music_id}'  # Return the embed URL
        
        # If still not found, try searching only by track name (may be a cover by another artist)
        query = f'track:"{nome_musica}"'  # Create a query with only the track name as last resort
        result = sp.search(q=query, limit=15, type='track')  # Search with more results (15) as it's a less specific search
        
        if result['tracks']['items']:  # Check if the track-only search returned results
            # Choose the first available result
            music_id = result['tracks']['items'][0]['id']  # Get the track ID from first result
            print(f"Song found (possibly a cover): {result['tracks']['items'][0]['name']} by {result['tracks']['items'][0]['artists'][0]['name']}")  # Log the found track
            return f'https://open.spotify.com/embed/track/{music_id}'  # Return the embed URL
        
        return 'Song not found'  # Return message if no tracks were found after all search attempts
    except Exception as e:  # Catch any exceptions that might occur during the search process
        print(f"Error searching for song '{nome_musica}': {e}")  # Log the error with the song name
        return 'Search error'  # Return error message


In [ ]:
df_songs = df_album_song.drop_duplicates(subset=['song'])  # Create a new DataFrame with only unique songs, removing duplicates based on the 'song' column
df_songs_spotify = df_songs[['song', 'song_upper']]  # Create a new DataFrame with only the 'song' and 'song_upper' columns from the unique songs DataFrame
df_songs_spotify['spotify_link'] = df_songs_spotify['song'].apply(obter_link_spotify)  # Apply the obter_link_spotify function to each song name, storing the returned Spotify embed links in a new column
df_songs_spotify['spotify_code'] = df_songs_spotify['spotify_link'].str.extract(r'track/([^/?]+)')  # Extract the Spotify track ID from the embed link using regex and store it in a new column

In [ ]:
# Merge the original album_song DataFrame with the Spotify information using 'song_upper' as the common key, keeping all rows from the left DataFrame (df_album_song) even if they don't have a match
df_album_song = pd.merge(df_album_song, df_songs_spotify[['song_upper', 'spotify_link', 'spotify_code']], on='song_upper', how='left')  

In [ ]:
 # Merge the setlist song data (result_song_id) with album information and Spotify links using 'song_upper' as the joining key; the 'left' join preserves all rows from result_song_id even if they don't have matching album data
result_merged_with_albums = pd.merge(result_song_id, df_album_song[['song_upper', 'album', 'year', 'spotify_link', 'spotify_code']], on='song_upper', how='left') 

Part III - Clean the Dataframe

In [ ]:
# List of the columns you want to keep
selected_columns = ['id', 'eventDate', 'info', 'url', 'artist.name', 'venue.name', 'venue.city.name', 'venue.city.state', 'venue.city.stateCode', 'venue.city.coords.lat', 'venue.city.coords.long', 'venue.city.country.code', 'venue.city.country.name', 'tour.name']

In [ ]:
# Filter the Dataframe with the selected colunms
filtered_df = df[selected_columns]

In [ ]:
filtered_df = filtered_df.rename(columns={'eventDate': 'date'})  # Rename the 'eventDate' column to the simpler 'date'
filtered_df = filtered_df.rename(columns={'artist.name': 'artist_name'})  # Replace the nested column name 'artist.name' with the flattened 'artist_name'
filtered_df = filtered_df.rename(columns={'venue.name': 'venue'})  # Simplify 'venue.name' to just 'venue'
filtered_df = filtered_df.rename(columns={'venue.city.name': 'venue_city'})  # Convert the deeply nested 'venue.city.name' to the more accessible 'venue_city'
filtered_df = filtered_df.rename(columns={'venue.city.state': 'venue_state'})  # Flatten 'venue.city.state' to 'venue_state'
filtered_df = filtered_df.rename(columns={'venue.city.stateCode': 'venue_state_code'})  # Change 'venue.city.stateCode' to the more consistent 'venue_state_code'
filtered_df = filtered_df.rename(columns={'venue.city.coords.lat': 'venue_lat'})  # Convert the geographic coordinate 'venue.city.coords.lat' to the simpler 'venue_lat'
filtered_df = filtered_df.rename(columns={'venue.city.coords.long': 'venue_long'})  # Rename longitude coordinate from 'venue.city.coords.long' to 'venue_long'
filtered_df = filtered_df.rename(columns={'venue.city.country.code': 'venue_country_code'})  # Simplify the country code column from nested format to flat 'venue_country_code'
filtered_df = filtered_df.rename(columns={'venue.city.country.name': 'venue_country_name'})  # Flatten the country name from 'venue.city.country.name' to 'venue_country_name'
filtered_df = filtered_df.rename(columns={'tour.name': 'tour_name'})  # Rename 'tour.name' to the more consistent 'tour_name'

In [ ]:
# Convert to datetime
filtered_df['data_datetime'] = pd.to_datetime(filtered_df['date'])

# Keep only the Date
filtered_df['event_date'] = filtered_df['data_datetime'].dt.date


In [ ]:
# Shift the latitude values down by one row to align the next event's latitude with the current row
filtered_df['venue_lat_next'] = filtered_df['venue_lat'].shift(+1)

# Shift the longitude values down by one row to align the next event's longitude with the current row
filtered_df['venue_long_next'] = filtered_df['venue_long'].shift(+1)


In [ ]:
# Define the columns that may contain missing geographic coordinates
columns_to_fill = ['venue_lat_next', 'venue_long_next', 'venue_lat', 'venue_long']

# Replace NaN values in these columns with 0 to avoid errors in further calculations (e.g., distance)
filtered_df[columns_to_fill] = filtered_df[columns_to_fill].fillna(0)

In [ ]:
# Create lists of coordinates by pairing latitude and longitude for origins and destinations
origins = list(zip(filtered_df['venue_lat'], filtered_df['venue_long']))
destinations = list(zip(filtered_df['venue_lat_next'], filtered_df['venue_long_next']))

# Calculate geodesic distance in miles between each pair of origin and destination coordinates
filtered_df['distance_miles'] = [geodesic(origin, destination).miles for origin, destination in zip(origins, destinations)]

In [ ]:
# Fill null values in the tour_name column with the event year (converted to string) from data_datetime
filtered_df['tour_name'] = filtered_df['tour_name'].fillna(filtered_df['data_datetime'].dt.year.astype(str))

In [ ]:
# Merge the filtered_df and result_merged_with_albums DataFrames
merged_df = pd.merge(
    filtered_df,                        # First DataFrame to merge
    result_merged_with_albums,         # Second DataFrame to merge
    on='id',                            # Merge based on the 'id' column
    how='outer',                        # Perform an outer join to keep all rows from both DataFrames
    suffixes=('_filtered', '_albums')  # Add suffixes to columns with the same name to distinguish their origin
)


In [ ]:
# Export the merged DataFrame to an Excel file named 'setlist_songsplayed_a7x.xlsx' with a sheet named "setlist" and no index column
merged_df.to_excel('setlist_songsplayed_a7x.xlsx', sheet_name="setlist", index=False)

PART IV – Building the Pantab data frame
Transforming collected data into Hyper tables for smooth analysis and magic

In [ ]:
# Path to the first Excel file
excel_path1 = r"C:\Users\usuario\Documents\TC_DEMO_25_VDS-main\setlist_songsplayed_a7x.xlsx"

# Name of the sheet to read from the Excel file
sheet_name1 = "setlist"

# Read the specified sheet from the Excel file into a DataFrame
df1 = pd.read_excel(excel_path1, sheet_name=sheet_name1)

In [ ]:
# Define the path where the Hyper file will be saved
hyper_a7x_setlist = r"C:\Users\usuario\Documents\TC_DEMO_25_VDS-main\a7x_setlist.hyper"

# Create a Hyper file from the DataFrame using Pantab and save it to the defined path
pt.frame_to_hyper(df1, hyper_a7x_setlist, table = sheet_name1)

# Print confirmation message with the location of the created Hyper file
print(f"[Pantab] Hyper file created at: {hyper_a7x_setlist}")

PART V – Tableau Server Client — Publishing to the Cloud
Dropping the dashboard into the wild — ready for the world to rock

In [ ]:
# Authenticate to Tableau Server using a Personal Access Token (PAT)
tableau_auth = TSC.PersonalAccessTokenAuth(TOKENNAME, TOKENVALUE, site_id=CONTENTURL)

# Create a Tableau Server object to interact with the server, using the latest server API version
server = TSC.Server(SERVER, use_server_version=True)


In [ ]:
with server.auth.sign_in(tableau_auth):  # Sign in to Tableau Server using the authentication object
    # Retrieve all projects available on the Tableau site
    all_project_items, pagination_item = server.projects.get()
    
    # Print a list of project IDs retrieved from the server
    print([proj.id for proj in all_project_items])


In [ ]:
# ---- Connect to Tableau CLoud and publish a data source ----
with server.auth.sign_in(tableau_auth):  # Sign in to Tableau Server using the authentication object
    
    # Specify the target project ID where the datasource will be published
    project_id = '5bc07b92-a197-460a-991d-020e906cc259'
    
    # Define the local path to the Hyper file to be published
    file_path_1 = r'C:\Users\usuario\Documents\TC_DEMO_25_VDS-main\a7x_setlist.hyper'

    # Create a new datasource item associated with the specified project
    new_datasource_1 = TSC.DatasourceItem(project_id)

    # Publish the Hyper file as a datasource to Tableau Server, overwriting if it already exists
    new_datasource_1 = server.datasources.publish(
                        new_datasource_1, file_path_1, 'Overwrite')
